In [70]:
import numpy as np
import random
import keras.backend as K
from faker import Faker
from babel.dates import format_date
from keras.utils import to_categorical
from keras.layers import RepeatVector, Concatenate, Dense, Activation, Dot
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.models import Model

In [54]:
random.seed(12345)

f = Faker()
f.seed(12345)


FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']


In [55]:
def gen_data():
    
    train_data = []
    human_vocab = set()
    machine_vocab = set()
    
    for i in range(10000):
        dt = f.date_object()
        h = format_date(dt, format=random.choice(FORMATS), locale='en_US').lower().replace(',','')
        m = dt.isoformat()
        if h is not None:
            train_data.append((h,m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    h_dict = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'],
                     list(range(len(human_vocab) + 2))))
    inverse_m_dict = dict(enumerate(sorted(machine_vocab)))
    m_dict = {i:w for w, i in inverse_m_dict.items()}

    return train_data, h_dict, m_dict, inverse_m_dict
    
        
    
 

In [56]:
train_data, h_dict, m_dict, inverse_m_dict = gen_data()

In [57]:
train_data[:10]

[('9 may 1998', '1998-05-09'),
 ('10.09.70', '1970-09-10'),
 ('4/28/90', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('tuesday july 8 2008', '2008-07-08'),
 ('08 sep 1999', '1999-09-08'),
 ('1 jan 1981', '1981-01-01'),
 ('monday may 22 1995', '1995-05-22')]

In [58]:
def string_to_int(s, max_len, vocab):
    
    s = s.lower().replace(',','')
    
    if len(s) > max_len:
        s = s[:max_len]
    
    rep = list(map(lambda x: vocab.get(x, '<unk>'), s))
    if len(s) < max_len:
        rep += [vocab['<pad>']] * (max_len - len(s))
        
    return rep
        
   
  

In [59]:
X_train, y_train = zip(*train_data)
max_len_x = 30
max_len_y = 10

X_train = np.array([string_to_int(i, max_len_x, h_dict) for i in X_train])
y_train = np.array([string_to_int(i, max_len_y, m_dict) for i in y_train])

X_train_in = np.array(list(map(lambda x: to_categorical(x, num_classes=len(h_dict)), X_train)))
y_train_in = np.array(list(map(lambda x: to_categorical(x, num_classes=len(m_dict)), y_train)))







In [60]:
print(X_train.shape, y_train.shape, X_train_in.shape, y_train_in.shape)

(10000, 30) (10000, 10) (10000, 30, 37) (10000, 10, 11)


In [61]:
def softmax(x, axis=1):
    
    num_dim = K.ndim(x)
    if num_dim == 2:
        return K.softmax(x)
    else:
        expn = K.exp(x - K.max(x, axis=axis, keepdims=True))
        a = expn / K.sum(expn, axis=axis, keepdims=True)
        return a
    
    
    

In [62]:
rep = RepeatVector(max_len_x)
concat = Concatenate(axis=-1)
densor1 = Dense(10, activation='tanh')
densor2 = Dense(1, activation='relu')
act = Activation(softmax)
dotor = Dot(axes = 1)


In [63]:
def attention_step(a, m):
    
    m = rep(m)
    c = concat([a, m])
    e = densor1(c)
    e = densor2(e)
    alphas = act(e)
    context = dotor([alphas, a])
    
    return context
    

In [64]:
a_size = 32
m_size = 64
post_activation_LSTM = LSTM(m_size, return_state=True)
ou_layer = Dense(len(m_dict), activation=softmax)






In [67]:
def model(max_len_x, max_len_y, a_size, m_size, h_dict_size, m_dict_size):
    
    X = Input(shape=(max_len_x, h_dict_size))
    m0 = Input(shape=(m_size,))
    c0 = Input(shape=(m_size,))
   
    outputs = []
    m = m0
    c = c0
    
    a = Bidirectional(LSTM(a_size, return_sequences=True))(X)
    
    for i in range(max_len_y):
        context = attention_step(a, m)
        m, _, c = post_activation_LSTM(context, initial_state=[m, c])
        out = ou_layer(m)
        outputs.append(out)
    
    m = Model(inputs=[X, m0, c0], outputs=outputs)
    
    return m
        

In [68]:
m = model(max_len_x, max_len_y, a_size, m_size, len(h_dict), len(m_dict))


In [69]:
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 30, 37)       0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 30, 64)       17920       input_13[0][0]                   
__________________________________________________________________________________________________
repeat_vector_3 (RepeatVector)  (None, 30, 64)       0           input_14[0][0]                   
                                                                 lstm_5[0][0]                     
          

In [71]:
optimizer = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
m.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [72]:
num_examples = 10000
m0 = np.zeros((num_examples, m_size))
c0 = np.zeros((num_examples, m_size))
outputs = list(y_train_in.swapaxes(0,1))



In [75]:
m.fit([X_train_in, m0, c0], outputs, epochs=1, batch_size=100)


Epoch 1/1
10000/10000 [==============================] - 48s 5ms/step - loss: 16.4251 - dense_8_loss_1: 1.2498 - dense_8_loss_2: 1.0554 - dense_8_loss_3: 1.7242 - dense_8_loss_4: 2.6729 - dense_8_loss_5: 0.7316 - dense_8_loss_6: 1.2265 - dense_8_loss_7: 2.5789 - dense_8_loss_8: 0.9391 - dense_8_loss_9: 1.6736 - dense_8_loss_10: 2.5731 - dense_8_acc_1: 0.4948 - dense_8_acc_2: 0.6575 - dense_8_acc_3: 0.3017 - dense_8_acc_4: 0.0753 - dense_8_acc_5: 0.9643 - dense_8_acc_6: 0.3811 - dense_8_acc_7: 0.0662 - dense_8_acc_8: 0.9067 - dense_8_acc_9: 0.2595 - dense_8_acc_10: 0.1050      